In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import KBinsDiscretizer
import time

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
from collections import Counter

cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"]
    # NumPumpsAttending??
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

def Create_X(df_limited, index):    
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = [item for sublist in cols_cible for item in sublist] + [
        "NumPumpsAttending"
    ]
    print("cols_to_remove", cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes
    X = X[cols_to_keep]

    display(X.head(2))
    print(X.shape)
    return X

# Simulation d'un RandomUnderSampler
def custom_random_undersampler(X_train, y_train, bins=25, random_state=42):
    print("Resampling")
    print(len(X_train))
    print(type(X_train), type(y_train))
    # Discrétisation de la cible continue
    y_binned, bin_edges = pd.cut(y_train, bins=bins, labels=False, retbins=True)
    print(bin_edges)
    print(y_binned.value_counts())
    # Application de RandomUnderSampler
    rus = RandomUnderSampler(sampling_strategy="majority", random_state=random_state)
    X_resampled, y_resampled_binned = rus.fit_resample(X_train, y_binned)
    print(y_resampled_binned.value_counts())
    # Récupérer les cibles originales à partir des bins
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    y_resampled = bin_midpoints[y_resampled_binned]
    print(y_resampled)
    print(len(X_resampled))
    return X_resampled, y_resampled

In [3]:
# Créez le cercle de corrélation
# def draw_correlation_circle(df_charges_factorielles, pca, arrow_length=0.1, label_rotation=0):
#     fig, ax = plt.subplots(figsize=(8, 8))
#     for i, var in enumerate(df_charges_factorielles.columns):
#         x = df_charges_factorielles.loc[0, var]
#         y = df_charges_factorielles.loc[1, var]
#         ax.arrow(0, 0, x, y, head_width=arrow_length, head_length=arrow_length, fc='gray', ec='gray')
#         ax.text(x*1.15, y*1.15, var, ha='center', va='center', rotation=label_rotation, fontsize=9)
#     circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='black')
#     ax.add_artist(circle)
#     ax.set_xlim(-1.1, 1.1)
#     ax.set_ylim(-1.1, 1.1)
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlabel('Axe 1 (PC1)')
#     ax.set_ylabel('Axe 2 (PC2)')
#     ax.set_title('Cercle des Corrélations')
#     plt.grid()
#     plt.show()

cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]

ignore_regressors = [
    "ExtraTreesRegressor",
    "NuSVR",
    "QuantileRegressor",
    "SVR",
    "CCA",
    "GammaRegressor",
    "GaussianProcessRegressor",
    "IsotonicRegression",
    "MultiOutputRegressor",
    "MultiTaskElasticNet",
    "MultiTaskElasticNetCV",
    "MultiTaskLasso",
    "MultiTaskLassoCV",
    "PLSCanonical",
    "RadiusNeighborsRegressor",
    "RegressorChain",
    "StackingRegressor",
    "KernelRidge",
    "ARDRegression",
    "TweedieRegressor",
]
REGRESSORS = [
    estimator
    for estimator in all_estimators()
    if (
        issubclass(estimator[1], RegressorMixin)
        and (not (estimator[0] in ignore_regressors))
    )
]

all_results = pd.DataFrame()

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    # X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    # display(X.head(2))
    # print(X.shape)

    # pca = PCA()
    # data_2D = pca.fit_transform(X)
    # display(data_2D)
    # print("La part de variance expliquée est", round(pca.explained_variance_ratio_.sum(), 5))

    # plt.figure()
    # plt.xlim(0,40)
    # plt.xlabel('Nombre de composantes')
    # plt.ylabel('Part de variance expliquée')
    # plt.axhline(y = 0.95, color ='r', linestyle = '--')
    # plt.plot(pca.explained_variance_ratio_.cumsum());
    # plt.show()

    # variance_expliquee = pca.explained_variance_ratio_
    # charges_factorielles = pca.components_
    # # Créez un DataFrame pour afficher les corrélations de chaque variable avec chaque axe
    # df_charges_factorielles = pd.DataFrame(
    #     charges_factorielles,
    #     columns=X.columns,
    #     #index=["Axe 1", "Axe 2", "Axe 3", "Axe 4", "Axe 5", "Axe 6"],
    # )

    # # Affichez le DataFrame des charges factorielles
    # display(df_charges_factorielles.head(2))

    # charges_factorielles = pca.components_
    # # Appelez la fonction pour tracer le cercle de corrélation
    # draw_correlation_circle(df_charges_factorielles, pca)

    # Auto ML
    for year_floor in range(13, 16):
        print("-----------------------------------------------------")
        print("year", 2024 - 16 + year_floor)
        print("-----------------------------------------------------")
        df_limited = df[df.CalYear >= year_floor]  # >= 2021
        X = Create_X(df_limited, index)

        for col_cible in cols_cible[index]:
            if not(col_cible in cols_cible_filter):
                continue
            print("cible", col_cible)
            y = df_limited[col_cible]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            # Réchantillonne
            # X_train_samp, y_train_samp = custom_random_undersampler(X_train, y_train)
            X_train_samp, y_train_samp = X_train, y_train
            # LazyPredict
            myREGRESSORS = REGRESSORS
            print(myREGRESSORS)
            # myREGRESSORS = myREGRESSORS[4:6]
            reg = LazyRegressor(
                verbose=2,
                ignore_warnings=False,
                custom_metric=None,
                regressors=myREGRESSORS,
            )
            models, predictions = reg.fit(X_train_samp, X_test, y_train_samp, y_test)
            # models, predictions = reg.fit(X_train, X_test, y_train, y_test)
            # Ajoute le nom de la colonne
            models["Target"] = col_cible
            models["Year floor"] = 2024 - 16 + year_floor
            # Concaténe dans les résultats
            all_results = pd.concat([all_results, models], axis=0)
            display(models)
            models.to_csv(f"../data/_autoML_regressor_{col_cible}.csv", sep=";", index=True)
            # break
        # break
all_results = all_results.sort_values(by=["Target", "RMSE"], ascending=False)
all_results.to_csv(f"../data/_autoML_regressor.csv", sep=";", index=True)

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

-----------------------------------------------------
year 2021
-----------------------------------------------------
(444247, 42)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
1147280,13,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
1147281,13,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0


(444247, 18)
cible TurnoutTimeSeconds_mean
[('AdaBoostRegressor', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>), ('BaggingRegressor', <class 'sklearn.ensemble._bagging.BaggingRegressor'>), ('BayesianRidge', <class 'sklearn.linear_model._bayes.BayesianRidge'>), ('DecisionTreeRegressor', <class 'sklearn.tree._classes.DecisionTreeRegressor'>), ('DummyRegressor', <class 'sklearn.dummy.DummyRegressor'>), ('ElasticNet', <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>), ('ElasticNetCV', <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>), ('ExtraTreeRegressor', <class 'sklearn.tree._classes.ExtraTreeRegressor'>), ('GradientBoostingRegressor', <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>), ('HistGradientBoostingRegressor', <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>), ('HuberRegressor', <class 'sklearn.linear_model._huber.HuberRegressor'>), ('KNeighborsRegressor', <class 'sklearn.neig

  3%|▎         | 1/35 [00:05<02:56,  5.18s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.11130882084114446, 'Adjusted R-Squared': 0.11112874360206282, 'RMSE': np.float64(34.86080349122066), 'Time taken': 5.182554721832275}


...

 91%|█████████▏| 32/35 [05:49<00:08,  2.82s/it]

{'Model': 'SGDRegressor', 'R-Squared': 0.13908706751844058, 'Adjusted R-Squared': 0.13883831511529643, 'RMSE': np.float64(1353.5571676547406), 'Time taken': 0.7563908100128174}


100%|██████████| 35/35 [06:39<00:00, 11.41s/it]

{'Model': 'TheilSenRegressor', 'R-Squared': 0.003514966166231037, 'Adjusted R-Squared': 0.0032270415254095353, 'RMSE': np.float64(1456.2380593240218), 'Time taken': 49.81485366821289}
{'Model': 'TransformedTargetRegressor', 'R-Squared': 0.13962181023631026, 'Adjusted R-Squared': 0.13937321234186395, 'RMSE': np.float64(1353.1367318352352), 'Time taken': 0.1827106475830078}
VotingRegressor model failed to execute
VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,Target,Year floor
Model,,,,,,
RandomForestRegressor,0.22,0.22,1288.40,9.48,PumpSecondsOnSite_mean,2023
HistGradientBoostingRegressor,0.22,0.22,1288.52,0.51,PumpSecondsOnSite_mean,2023
BaggingRegressor,0.22,0.22,1288.90,0.89,PumpSecondsOnSite_mean,2023
MLPRegressor,0.22,0.22,1288.91,162.97,PumpSecondsOnSite_mean,2023
DecisionTreeRegressor,0.22,0.22,1289.21,0.22,PumpSecondsOnSite_mean,2023
ExtraTreeRegressor,0.22,0.22,1289.89,0.25,PumpSecondsOnSite_mean,2023
GradientBoostingRegressor,0.21,0.21,1293.53,4.79,PumpSecondsOnSite_mean,2023
AdaBoostRegressor,0.16,0.16,1335.07,1.09,PumpSecondsOnSite_mean,2023
BayesianRidge,0.14,0.14,1353.14,0.24,PumpSecondsOnSite_mean,2023


In [ ]:
# Regressors retenus pour optimisation, prend le top 5 de chaque cible et les entraine tous
# pour voir si au final on peut limiter à certains communs

# Pour TurnoutTimeSeconds
# HistGradientBoostingRegressor
# MLPRegressor
# GradientBoostingRegressor
# DecisionTreeRegressor
# ExtraTreeRegressor

# Pour TravelTimeSeconds
# MLPRegressor
# HistGradientBoostingRegressor
# GradientBoostingRegressor
# LassoCV
# RidgeCV

# Pour PumpSecondsOnSite
# HistGradientBoostingRegressor
# MLPRegressor
# RandomForestRegressor
# BaggingRegressor
# DecisionTreeRegressor

# On va effectuer un GridSearchCV pour :
# HistGradientBoostingRegressor
# MLPRegressor
# GradientBoostingRegressor
# DecisionTreeRegressor
# ExtraTreeRegressor
# LassoCV
# RidgeCV
# RandomForestRegressor
# BaggingRegressor


import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
    BaggingRegressor,
    AdaBoostRegressor,
)
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import (
    PoissonRegressor,
    Lars,
    LarsCV,
    LinearRegression,
    Ridge,
    RidgeCV,
    BayesianRidge,
    LassoLarsIC,
    LassoCV,
    LassoLarsCV,
    ElasticNetCV,
    OrthogonalMatchingPursuitCV,
    PassiveAggressiveRegressor,
    RANSACRegressor,
    SGDRegressor,
    HuberRegressor,
    TheilSenRegressor,
    ElasticNet,
    OrthogonalMatchingPursuit,
    LassoLars,
    Lasso,
)
from sklearn.svm import LinearSVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import TransformedTargetRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold

from sklearn.metrics import accuracy_score, root_mean_squared_error, mean_absolute_error


# Hyperparamètres larges pour chaque modèle
param_grid = {
    "HistGradientBoostingRegressor": {
        "learning_rate": [0.01, 0.05, 0.1],
        "max_iter": [100, 200, 300],
        "max_depth": [None, 5, 10, 15, 20],
    },
    "MLPRegressor": {
        "hidden_layer_sizes": [(50,), (100,), (50, 50), (100, 100)],
        "activation": ["relu", "tanh", "logistic"],
        "solver": ["adam", "sgd"],
        "learning_rate_init": [0.0001, 0.001, 0.01, 0.1],
    },
    "GradientBoostingRegressor": {
        "learning_rate": [0.01, 0.1, 0.2],
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 5, 7],
    },
    "DecisionTreeRegressor": {
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    },
    "ExtraTreeRegressor": {
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    },
    "RandomForestRegressor": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
    },
    "BaggingRegressor": {
        "n_estimators": [10, 50, 100],
        "max_samples": [0.5, 0.75, 1.0],
        "max_features": [0.5, 1.0],
    },
    "AdaBoostRegressor": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.1, 0.5, 1.0],
    },
    "PoissonRegressor": {
        "alpha": [0.01, 0.1, 1.0],
    },
    "Lars": {
        "n_nonzero_coefs": [1, 5, 10, 20],
    },
    "LinearRegression": {},
    "TransformedTargetRegressor": {},
    "Ridge": {
        "alpha": [-5, 0.1, 1.0, 10.0, 50.0, 100.0],
        "fit_intercept": [True, False],
    },
    "BayesianRidge": {
        "alpha_1": [1e-6, 1e-5, 1e-4],
        "alpha_2": [1e-6, 1e-5, 1e-4],
    },
    "LassoLarsIC": {"criterion": ["aic", "bic"]},
    "OrthogonalMatchingPursuitCV": {},
    "PLSRegression": {
        "n_components": [2, 5, 10],
    },
    "TheilSenRegressor": {
        "max_subpopulation": [10000, 100000],
    },
    "HuberRegressor": {
        "alpha": [0.01, 0.1, 1.0],
    },
    "LinearSVR": {
        "C": [0.1, 1.0, 10.0],
        "epsilon": [0.1, 0.2, 0.3],
    },
    "Lasso": {
        "alpha": [0.1, 1.0, 5.0, 10.0],
    },
    "ElasticNet": {
        "alpha": [0.1, 1.0, 10.0],
        "l1_ratio": [0.1, 0.5, 0.7, 0.9],
    },
    "OrthogonalMatchingPursuit": {},
    "KNeighborsRegressor": {
        "n_neighbors": [3, 5, 10, 15],
        "weights": ["uniform", "distance"],
    },
    "PassiveAggressiveRegressor": {
        "C": [0.1, 1.0, 10.0],
    },
    "RANSACRegressor": {"min_samples": [0.1, 0.5, 0.75], "max_trials": [100, 200, 300]},
    "SGDRegressor": {
        "alpha": [0.0001, 0.001, 0.01],
        "penalty": ["l2", "l1", "elasticnet"],
    },
}

# Liste des modèles
regressors = {
    "HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    "MLPRegressor": MLPRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "BaggingRegressor": BaggingRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "PoissonRegressor": PoissonRegressor(),
    "Lars": Lars(),
    "LinearRegression": LinearRegression(),
    "TransformedTargetRegressor": TransformedTargetRegressor(
        regressor=LinearRegression()
    ),
    "Ridge": Ridge(),
    "BayesianRidge": BayesianRidge(),
    "LassoLarsIC": LassoLarsIC(),
    "OrthogonalMatchingPursuitCV": OrthogonalMatchingPursuitCV(),
    "PLSRegression": PLSRegression(),
    "TheilSenRegressor": TheilSenRegressor(),
    "HuberRegressor": HuberRegressor(),
    "LinearSVR": LinearSVR(),
    "LassoLars": LassoLars(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "OrthogonalMatchingPursuit": OrthogonalMatchingPursuit(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
    "RANSACRegressor": RANSACRegressor(),
    "SGDRegressor": SGDRegressor(),
}

results = []
crossval = KFold(n_splits=5, random_state=42, shuffle=True)

for index, name in enumerate(cols_cible_type):
    df = load_df(name)

    for year_floor in range(13, 16):        
        print("-----------------------------------------------------")
        print("year", 2024 - 16 + year_floor)
        print("-----------------------------------------------------")
        df_limited = df[df.CalYear >= year_floor]  # >= 2021
        X = Create_X(df_limited, index)
        for index_cible, col_cible in enumerate(cols_cible[index]):
            print(
                "--------------------------------------------------------------------------------"
            )
            print("///////////////////////////", "cible", col_cible)
            y = df_limited[col_cible]
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )
            # Réchantillonne
            # X_train_smote, y_train_smote = under_sampling(X_train, y_train)
            X_train_samp = X_train
            y_train_samp = y_train

            # Exécuter GridSearchCV pour chaque modèle
            for model_name, model in regressors.items():
                # Controle run, car potentiellement long
                print("-------------------------------------------")
                print(f"GridSearchCV pour {model_name}")
                grid = GridSearchCV(
                    model,
                    param_grid[model_name],
                    scoring="neg_mean_squared_error",
                    cv=crossval,
                    n_jobs=-1,
                    verbose=2,
                )
                # Fit et Predict
                grid.fit(X_train_samp, y_train_samp)
                y_pred = grid.predict(X_test)
                y_train_pred = grid.predict(X_train_samp)
                # Meilleur score et meilleurs hyperparamètres
                best_params = grid.best_params_
                best_score = grid.best_score_
                # affiche de suite les résultat
                print(f"Meilleurs paramètres {best_params}")
                print(f"Meilleur score {best_score}")
                # Accuracy
                accuracy = accuracy_score(y_test, y_pred)
                print(f"Accuracy : {accuracy}")
                # train et test RMSE
                score_train = grid.score(X_train_samp, y_train_samp)
                print(f"Score train : {score_train}")
                score_test = grid.score(X_test, y_test)
                print(f"Score test : {score_test}")
                # RMSE
                rmse_train = root_mean_squared_error(y_train_samp, y_train_pred)
                print(f"RMSE train : {rmse_train}")
                rmse_test = root_mean_squared_error(y_test, y_pred)
                print(f"RMSE test : {rmse_test}")
                # MAE
                mae_train = mean_absolute_error(y_train_samp, y_train_pred)
                print(f"MAE train : {score_train}")
                mae_test = mean_absolute_error(y_test, y_pred)
                print(f"MAE test : {score_test}")

                # Score de validation croisée
                #             scores = cross_validate(grid, X_train, y_train, cv=crossval, scoring=['accuracy', 'precision', 'recall', 'f1'])
                #             cv_test_accuracy = f" cv_test_accuracy : {scores['test_accuracy'].mean()} (+/-{scores['test_accuracy'].std()})"
                #             cv_test_precision
                #             cv_test_recall
                #             cv_test_f1

                # {'fit_time': array([1.7602632 , 1.3521595 , 1.53327417, 1.6628437 , 1.52980328]),
                #  'score_time': array([0.10070086, 0.10009599, 0.11566949, 0.09443331, 0.09455991]),
                #  'test_accuracy': array([0.87765306, 0.87768222, 0.8774344 , 0.87757839, 0.87759297]),
                #  'test_precision': array([0.36      , 0.43243243, 0.2745098 , 0.29032258, 0.38888889]),
                #  'test_recall': array([0.00107322, 0.00190794, 0.00166965, 0.00107335, 0.00250447]),
                #  'test_f1': array([0.00214005, 0.00379912, 0.00331911, 0.00213878, 0.00497689])}

                # print("[StackingClassifier]: \n Accuracy: %0.2f (+/- %0.2f)\n" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
                #       "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))
                # >>>
                # [StackingClassifier]:
                #  Accuracy: 0.76 (+/- 0.02)
                #  F1 score: 0.62 (+/- 0.04)

                # Ajouter au tableau des résultats
                results.append(
                    {
                        "Target": col_cible,
                        "Year floor": 2024 - 16 + year_floor,
                        "Model": model_name,
                        "Best Params": best_params,
                        "Best Score": best_score,
                        "Accuracy Test": accuracy,
                        "Score Train": score_train,
                        "Score Test": score_test,
                        "RMSE train": rmse_train,
                        "RMSE test": rmse_test,
                        "MAE train": score_train,
                        "MAE test": score_test,
                    }
                )
print(results)
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by=["Target", "MAE test"], ascending=False)
df_results.to_csv(f"../data/_GridSearch_regressor.csv", sep=";", index=True)

# p = plt.figure(figsize=(18,12))
# p = sns.set_theme(style="white")
# p = models=models.sort_values(by='Model Accuracy with GridSearch',ascending=False)
# p = sns.barplot(y= 'Model', x= 'Model Accuracy with GridSearch', data= models)
# for container in p.containers:
#     p.bar_label(container,label_type = 'edge',padding = 8,size = 20,color = "black",rotation = 0,
#     bbox={"boxstyle": "round", "pad": 0.6, "facecolor": "pink", "edgecolor": "Blue", "alpha": 1})
# plt.title('COMPARE THE MODEL')
# plt.xlabel('MODEL')
# plt.ylabel('Model Accuracy')
# plt.show()

In [ ]:

StackingRegressor

from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Modèles de base
base_models = [
    ('linear', LinearRegression()),
    ('tree', DecisionTreeRegressor(max_depth=5)),
    ('forest', RandomForestRegressor(n_estimators=50))
]

# Modèle final
final_model = LinearRegression()

# Stacking Regressor
stacking = StackingRegressor(estimators=base_models, final_estimator=final_model, cv=5)

# Entraînement
stacking.fit(X_train, y_train)

# Prédictions
y_pred = stacking.predict(X_test)

Voting Regressor

from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Modèles
models = [
    ('ridge', Ridge(alpha=1.0)),
    ('tree', DecisionTreeRegressor(max_depth=5)),
    ('gboost', GradientBoostingRegressor(n_estimators=50))
]

# Voting Regressor
voting = VotingRegressor(estimators=models)

# Entraînement
voting.fit(X_train, y_train)

# Prédictions
y_pred = voting.predict(X_test)


XGBoostRegressor

In [ ]:
lancer regressor et classifiers et faire voting dessus